In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_excel('../Jaehoon/fifinal_merge_data.xlsx')
data

,Unnamed: 0,경주일자,경주번호,경주거리,착순,출주번호,마필연령,마필성별,경주기록(초),부담중량,...,"(360,)","(800,)","(900,)","(1000,)","(1110,)","(1200,)","(1400,)","(1610,)","(1700,)","(1800,)"
0,0,20230624,1,800,1,1,-1.671258,-1,68.1,-0.256074,...,0,1,0,0,0,0,0,0,0,0
1,1,20230624,1,800,2,4,0.185695,1,68.5,-0.896258,...,0,1,0,0,0,0,0,0,0,0
2,2,20230624,1,800,3,9,-1.671258,0,68.5,0.384111,...,0,1,0,0,0,0,0,0,0,0
3,3,20230624,1,800,4,3,0.185695,0,68.6,1.024295,...,0,1,0,0,0,0,0,0,0,0
4,4,20230624,1,800,5,8,0.185695,0,69.0,0.384111,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28678,28678,20190111,9,900,6,8,-0.654654,1,64.1,0.098853,...,0,0,1,0,0,0,0,0,0,0
28679,28679,20190111,9,900,7,3,1.527525,1,64.2,1.746409,...,0,0,1,0,0,0,0,0,0,0
28680,28680,20190111,9,900,8,1,-0.654654,1,64.5,-0.889680,...,0,0,1,0,0,0,0,0,0,0
28681,28681,20190111,9,900,9,4,1.527525,1,64.6,-0.230658,...,0,0,1,0,0,0,0,0,0,0


In [3]:
data=data.drop(columns=['Unnamed: 0'])
data=data.drop(columns=['제주4코너통과기록','조교사경력(년)','기수최근1년간1착승률','기수최근1년간2착승률','조교사최근1년간1착승률','조교사최근1년간2착승률','조교사위탁경주마1위승률','조교사위탁경주마2위승률'])
data=data.drop(columns=['조교사위탁경주마출전횟수','조교사최근1년간출전횟수'])
data['기수통산승률']=data['기수통산1착승률']+data['기수통산2착승률']
data['조교사통산승률']=data['조교사통산1착승률']+data['조교사통산2착승률']
data=data.drop(columns=['기수통산1착승률','기수통산2착승률','조교사통산1착승률','조교사통산2착승률'])
data=data.drop(columns=['3시간신적설(cm)'])
data=data.drop(columns=['mom_total_in3','dad_total_in3','mom_total','dad_total','dad_post','mom_post'])
data=data.drop(columns=['mom_rating','dad_rating'])
data=data.drop(columns=['경주거리'])

In [16]:
unique = data[["경주일자",	"경주번호"]].drop_duplicates()
unique = unique.reset_index()
unique

In [14]:
data["key"]=0
for i in range(len(unique)):
    data.loc[(data["경주일자"]==unique["경주일자"][i])&(data["경주번호"]==unique["경주번호"][i]), "key"] = i

array([  0,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 156, 157, 158, 161], dtype=int64)

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
X_train = drop_data[data["경주일자"]<20220701].drop(["경주기록(초)", "key"], axis=1)
X_test = drop_data[data["경주일자"]>=20220701].drop(["경주기록(초)", "key"], axis=1) 
y_train = drop_data[data["경주일자"]<20220701]["경주기록(초)"]
y_test = drop_data[data["경주일자"]>=20220701]["경주기록(초)"]

params = {
    'max_depth':5,
    'reg_lambda':1,
    'learning_rate': 0.1,
    'n_estimators': 100
}
xgb_reg = XGBRegressor(params = params)
xgb_reg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_reg.predict(X_test)


In [ ]:
pred_key = pd.concat([drop_data[data["경주일자"]>=20220701]["key"], pd.DataFrame(y_pred)], axis=1)
pred_key.columns=["key",	"y_pred"]

true_key = pd.concat([drop_data[data["경주일자"]>=20220701]["key"], pd.DataFrame(y_test)], axis=1)
true_key.columns=["key",	"y_test"]

In [ ]:
pred_key["in5"] = 0
for i in pred_key["key"].unique():
    index = pred_key[pred_key["key"]==i].sort_values(by="y_pred", ascending=True)[:5].index
    pred_key.loc[index,"in5"]=1

In [ ]:
true_key["in3"] = 0
for i in true_key["key"].unique():
    index = true_key[true_key["key"]==i].sort_values(by="y_test", ascending=True)[:3].index
    true_key.loc[index,"in3"]=1

In [ ]:
count = 0
for key in true_key["key"].unique():
    num = sum(true_key.loc[true_key["key"]==key,"in3"]&pred_key.loc[pred_key["key"]==key,"in5"])
    if num==3:
        count+=1

In [ ]:
count/len(true_key["key"].unique())